# 🛠️ Notebook: Einführung in Function Calling

In dem Notebook lernen wir, wie man LLMs dazu bringt, Funktionen aufzurufen.

## 📚 Quellen

- [OpenAI: Function Calling API](https://platform.openai.com/docs/guides/function-calling)
- [Ollama: Function Calling](https://ollama.com/blog/functions-as-tools)

---

Viel Erfolg beim Ausprobieren der Function Calling Features! 🤗

Beginnen wir damit das Python Packet von ollama zu installieren. Anders als bei den vorherigen Notebooks, wo wir die OpenAI API genutzt haben, verwenden wir in diesem Notebook zur Abwechslung mal die Ollama API.


So wie `ollama`, bietet auch die Bibliothek `openai` die Möglichkeit, Requests an einen Server mit einer LLM-API zu senden. Beachte: `openai` unterstützt ebenso wie `ollama` Function Calling.

[https://platform.openai.com/docs/guides/function-calling](https://platform.openai.com/docs/guides/function-calling)

In [1]:
%%capture
!uv add ollama

In [2]:
LLM_URL = "http://132.199.138.16:11434"
# Nicht jedes LLM unterstützt Function Calling. 
# Gemma3, das LLM aus der letzten Übung wurde beispielsweise nicht dafür trainiert. Das open-source LLM "gpt-oss:20b" von OpenAI hingegen schon.
LLM_MODEL = "gpt-oss:20b"

In [3]:
from ollama import Client

client = Client(
  host=LLM_URL
)

### 1. Beispiel: Tool Call mit Aktienkursen

Definieren wir zunächst eine Liste mit Tools. In unserem Beispiel nur **ein** Tool, das den aktuellen Aktienkurs für ein gegebenes Symbol zurückgibt. 

Die `openai`-Bibliothek erwartet jedes Tool in einem bestimmten Format. Wir definieren das Tool als Dictionary mit folgenden Schlüsseln:

* `type`: OpenAI bietet prinzipiell auch andere Tool-Typen an (z.B. `web_search`), wir verwenden hier aber nur `function`.
* `name`: Der Name des Tools.
* `parameters`: Die Parameter, die die Funktion erwartet.
* `required`: Welche Parameter zwingend übergeben werden müssen.

In [4]:
tools = [{
    'type': 'function',
    'function': {
            'name': 'get_stock_price', # Name des Tools
            'description': 'Get the current stock price for a company', # Beschreibung des Tools
            'parameters': {
                'type': 'object',
                'properties': {
                    'symbol': { # Die Funktion erwartet genau einen Parameter, "symbol". Symbol ist das Börsenkürzel der Aktie, z.B. "AAPL" für Apple.
                        'type': 'string', # Typ des Parameters
                    },
                },
                'required': ['symbol'], # Erforderliche Parameter
            },
    },
}]

In [5]:
# Starten wir mit einem Beispiel, in dem wir den aktuellen Aktienkurs von SAP abfragen.
response = client.chat(
    model=LLM_MODEL,
    messages=[{'role': 'user', 'content': 'Wie steht die aktuelle SAP Aktie?'}], # Benutzeranfrage
    tools=tools, # Wir übergeben die Tools, die das Modell verwenden kann
)

# Wir können uns nun anschauen, ob und wie das Modell das Tool aufgerufen hat.
tool_calls = response['message']['tool_calls']
print(f"Tool calls: {tool_calls}")

# Argumente des Tool Calls und Funktion ausgeben
print(f"Function name: {tool_calls[0].function.name}")
print(f"Function arguments: {tool_calls[0].function.arguments}")

Tool calls: [ToolCall(function=Function(name='get_stock_price', arguments={'symbol': 'SAP'}))]
Function name: get_stock_price
Function arguments: {'symbol': 'SAP'}


In [6]:
# Wir können probeweise mal eine irrelevante Frage stellen, die nichts mit Aktien zu tun hat.
# In dem Fall ist tool_calls None, da das LLM kein Tool aufrufen muss.
response = client.chat(
    model=LLM_MODEL,
    messages=[{'role': 'user', 'content': 'Was ist 1+1?'}],
    tools=tools,
)

# Schauen wir uns die Ausgabe an. Bei der Ausgabe sehen wir, dass tool_calls None ist.
print(response)

model='gpt-oss:20b' created_at='2025-11-25T19:07:41.864122Z' done=True done_reason='stop' total_duration=659379875 load_duration=143220666 prompt_eval_count=131 prompt_eval_duration=48156583 eval_count=46 eval_duration=455991580 message=Message(role='assistant', content='1\u202f+\u202f1\u202f=\u202f2.', thinking='User asks: "Was ist 1+1?" Means "What is 1+1?" The answer is 2.', images=None, tool_name=None, tool_calls=None)


## Übungsaufgabe: Tool Execution - Funktionen tatsächlich ausführen

Bislang haben wir nur implementiert, dass das LLM ein Tool ggf. mit Parametern aufruft, aber die eigentliche Funktion dahinter wird nicht ausgeführt.

Implementieren wir nun mal eine kleine Pipeline, die tatsächlich eine Funktion ausführt, wenn das LLM ein Tool aufruft. Wir implementieren jetzt ein Tool, das Informationen über Länder abruft.

Gehen Sie für die Implementierung wie folgt vor:

1. Definieren Sie ein Tool `get_country_info` im JSON Schema mit Parameter `country`
2. Implementieren Sie die Funktion `get_country_info_from_api`, die Länder-Infos von der API abruft
3. Bauen Sie einen einfachen Workflow, der das LLM fragt und dann die Funktion aufruft


So funktioniert die API:

In [7]:
import requests

country = "Germany"  # Auch möglich: France, Italy, Japan, ...
url = f"https://restcountries.com/v3.1/name/{country}"
response = requests.get(url)
data = response.json()

# API gibt eine Liste zurück, wir nehmen das erste Element
country_data = data[0]
print(f"Name: {country_data['name']['common']}")
print(f"Hauptstadt: {country_data['capital'][0]}")
print(f"Bevölkerung: {country_data['population']}")
print(f"Region: {country_data['region']}")

# Schreiben Sie hier Ihren Code ...

Name: Germany
Hauptstadt: Berlin
Bevölkerung: 83491249
Region: Europe


<details>
<summary><b>Lösung anzeigen</b></summary>

```python
# 1. Tool Definition
tools = [{
    'type': 'function',
    'function': {
        'name': 'get_country_info',
        'description': 'Ruft Informationen über ein Land ab',
        'parameters': {
            'type': 'object',
            'properties': {
                'country': {
                    'type': 'string',
                    'description': 'Der Name des Landes (z.B. Germany, France, Japan)'
                }
            },
            'required': ['country']
        }
    }
}]

# 2. API-Funktion
def get_country_info_from_api(country):
    url = f"https://restcountries.com/v3.1/name/{country}"
    response = requests.get(url)
    data = response.json()[0]

    # Wir können die Informationen zu einem Text zusammenfassen:
    info = f"Land: {data['name']['common']}, Hauptstadt: {data['capital'][0]}, Bevölkerung: {data['population']}, Region: {data['region']}"
    return info


# 3. Workflow
messages = [{'role': 'user', 'content': 'Erzähl mir etwas über Norwegen.'}]

# LLM fragen
response = client.chat(model=LLM_MODEL,
                       messages=messages,
                       tools=tools,
                       options={"temperature": 0.0})

# Wurde ein Tool aufgerufen? Prüfen wir, ob der Key ['message']['tool_calls'] existiert
if response['message']['tool_calls']:
    country = response['message']['tool_calls'][0]['function']['arguments']['country']

    # Funktion ausführen
    country_info = get_country_info_from_api(country)

    # Prompt erstellen für finale Antwort
    prompt = f"Hier sind die Informationen über {country}:\n{country_info}.\nBitte fasse diese Informationen in zwei Sätzen kurz zusammen."

    # Zurück ans LLM
    messages = [{'role': 'user', 'content': prompt}]
    final_response = client.chat(
        model=LLM_MODEL, messages=messages, options={"temperature": 0.0})

    print(f"\nFinale Antwort: {final_response['message']['content']}")
```

</details>